In [1]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
df = pd.read_csv('big_df.csv')

In [3]:
df.head()

,frame0pixel0,frame0pixel1,frame0pixel2,frame0pixel3,frame0pixel4,frame0pixel5,frame0pixel6,frame0pixel7,frame0pixel8,frame0pixel9,...,frame14pixel391,frame14pixel392,frame14pixel393,frame14pixel394,frame14pixel395,frame14pixel396,frame14pixel397,frame14pixel398,frame14pixel399,type
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand


In [5]:
df.describe()

,frame0pixel0,frame0pixel1,frame0pixel2,frame0pixel3,frame0pixel4,frame0pixel5,frame0pixel6,frame0pixel7,frame0pixel8,frame0pixel9,...,frame14pixel390,frame14pixel391,frame14pixel392,frame14pixel393,frame14pixel394,frame14pixel395,frame14pixel396,frame14pixel397,frame14pixel398,frame14pixel399
count,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.000000,...,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.0,1953.0,1953.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001024,...,0.113671,0.084485,0.041475,0.009729,0.003584,0.000512,0.000512,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031993,...,0.317493,0.278186,0.199436,0.098178,0.059776,0.022628,0.022628,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0


In [6]:
X = df.drop(['type'], axis=1)
y = df['type']
y

0       backhand
1       backhand
2       backhand
3       backhand
4       backhand
          ...   
1948       smash
1949       smash
1950       smash
1951       smash
1952       smash
Name: type, Length: 1953, dtype: object

In [7]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([ 0,  0,  0, ..., 11, 11, 11])

In [9]:
np.arrayX.iloc[0][0:400]

frame0pixel0      0
frame0pixel1      0
frame0pixel2      0
frame0pixel3      0
frame0pixel4      0
                 ..
frame0pixel395    0
frame0pixel396    0
frame0pixel397    0
frame0pixel398    0
frame0pixel399    0
Name: 0, Length: 400, dtype: int64

In [8]:
frames = []
for i in range(0, df.shape[0]):
    for j in range(0, 400, 6000):
        frame = X.iloc[i][j:j+400]
        frame = frame.reshape(20, 20)
        frames.append(frame)
frames = np.array(frames)
frames

AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        # Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        # BatchNormalization(),
        # MaxPooling2D(pool_size=(2, 2)),
        # Dropout(0.2),
        
        Flatten(),
        # Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()